In [1]:
import numpy as np
import scipy.sparse as sp
import math
import pandas as pd

In [2]:
l09 = - math.log10(0.9)
l05 = - math.log10(0.5)

core_cube_gat2016 = np.zeros((3, 3, 6))
# werewolf might not vote possessed
core_cube_gat2016[1, 2, 0] = l05
# possessed might not vote werewolf
core_cube_gat2016[2, 1, 0] = l09
# werewolf would not vote werewolf
core_cube_gat2016[1, 1, 0] = 1.0
# possessed might not divine werewolf as werewolf
core_cube_gat2016[2, 1, 2] = l09
# werewolf might not divine possessed as werewolf
core_cube_gat2016[1, 2, 2] = l05
# werewolf would not divine werewolf as werewolf
core_cube_gat2016[1, 1, 2] = 1.0
# village Seer should not tell a lie
core_cube_gat2016[0, 0, 2] = 2.0
core_cube_gat2016[0, 2, 2] = 2.0
core_cube_gat2016[0, 1, 1] = 2.0
# village Medium should not tell a lie
core_cube_gat2016[0, 0, 4] = 2.0
core_cube_gat2016[0, 2, 4] = 2.0
core_cube_gat2016[0, 1, 3] = 2.0
# village Bodyguard should not tell a lie
# core_cube_gat2016[0, 1, 5] = 2.0

core_cube_gat2016 = -np.log(10)*core_cube_gat2016

In [3]:
class EnemyCaseTensor(object):
    """
    X_3d
    [i, j, 0] : agent i voted agent j (not in talk, action)
    [i, j, 1] : agent i divined agent j HUMAN
    [i, j, 2] : agent i divined agent j WEREWOLF
    [i, j, 3] : agent i inquested agent j HUMAN
    [i, j, 4] : agent i inquested agent j WEREWOLF
    [i, j, 5] : agent i managed to guard agent j
    X_2d
    [i, 0] : agent i is executed
    [i, 1] : agent i is attacked
    [i, 2] : agent i comingout himself/herself SEER
    [i, 3] : agent i comingout himself/herself MEDIUM
    [i, 4] : agent i comingout himself/herself BODYGUARD
    [i, 5] : agent i comingout himself/herself VILLAGER
    [i, 6] : agent i comingout himself/herself POSSESSED
    [i, 7] : agent i comingout himself/herself WEREWOLF
    """
    def __init__(self, core_cube_base, vilsize=15, n_wolves=3, n_features_3d=6, n_features_2d=8):
        # input parameters
        self.vilsize = vilsize
        self.n_wolves = n_wolves
        if self.n_wolves == 3:
            self.n_cases = vilsize * (vilsize - 1) * (vilsize - 2) * (vilsize - 3) / 6
        elif self.n_wolves == 2:
            self.n_cases = vilsize * (vilsize - 1) * (vilsize - 2) / 2
        else:
            raise ValueError('n_wolves must be 3 or 2')
        self.n_features_3d = n_features_3d
        self.n_features_2d = n_features_2d
        # info for the game
        self.X_3d = np.zeros((self.vilsize, self.vilsize, self.n_features_3d))
        self.X_2d = np.zeros((self.vilsize, self.n_features_2d))
        self.F_2d = np.zeros((self.n_cases, 3, self.n_features_2d))
        self.alive = np.zeros((self.n_cases, 3))
        # prob for cases
        self.Y_3d = np.zeros(self.n_cases)
        self.Y_2d = np.zeros(self.n_cases)
        self.Z_2d = np.ones(self.n_cases)
        # linear maps represent v/w/p for each cases, fixed, heavy to initailize
        self.T_2d = np.zeros((self.n_cases, 3, self.vilsize), dtype="int")
        self.T_4d = np.zeros((self.n_cases, 3, 3, self.vilsize, self.vilsize), dtype="int")
        # model parameter cube for X_3d in shape (3, 3, self.vilsize, self.vilsize, self.n_features_3d)
        self.core_cube = np.tensordot(
            np.ones((self.vilsize, self.vilsize, 1)), 
            core_cube_base.reshape((1, 3, 3, self.n_features_3d)), 
            axes=(2, 0)
        ).transpose((2, 3, 0, 1, 4))
        
    def initialize_X(self):
        self.X_3d = np.zeros((self.vilsize, self.vilsize, self.n_features_3d))
        self.X_2d = np.zeros((self.vilsize, self.n_features_2d))
    
    def initialize_T_2d(self):
        # make Tensor
        self.T_2d[:, 0, :] = 1
        if self.n_wolves == 3:
            ind = 0
            for w1 in range(self.vilsize - 2):
                for w2 in range(w1 + 1, self.vilsize - 1):
                    for w3 in range(w2 + 1, self.vilsize):
                        for p in range(self.vilsize):
                            if p not in [w1, w2, w3]:
                                self.T_2d[ind, 0, w1] = 0
                                self.T_2d[ind, 1, w1] = 1
                                self.T_2d[ind, 0, w2] = 0
                                self.T_2d[ind, 1, w2] = 1
                                self.T_2d[ind, 0, w3] = 0
                                self.T_2d[ind, 1, w3] = 1
                                self.T_2d[ind, 0, p] = 0
                                self.T_2d[ind, 2, p] = 1
                                ind += 1
        elif self.n_wolves == 2:
            ind = 0
            for w1 in range(self.vilsize - 1):
                for w2 in range(w1 + 1, self.vilsize):
                    for p in range(self.vilsize):
                        if p not in [w1, w2]:
                            self.T_2d[ind, 0, w1] = 0
                            self.T_2d[ind, 1, w1] = 1
                            self.T_2d[ind, 0, w2] = 0
                            self.T_2d[ind, 1, w2] = 1
                            self.T_2d[ind, 0, p] = 0
                            self.T_2d[ind, 2, p] = 1
                            ind += 1
        else:
            pass
        
    def initialize_T_4d(self):    
        for ind in range(self.n_cases):
            self.T_4d[ind, :, :, :, :] = np.tensordot(
                self.T_2d[ind, :, :].reshape((3, self.vilsize, 1)), 
                self.T_2d[ind, :, :].reshape((1, 3, self.vilsize)), 
                axes=(2, 0)
            ).transpose((0, 2, 1, 3))
        
    def update_Y(self):
        self.Y_3d = np.tensordot(
            self.T_4d, self.core_cube*np.tensordot(
                np.ones((3, 3, 1)), 
                self.X_3d.reshape((1, self.vilsize, self.vilsize, self.n_features_3d)), axes=(2, 0)
            ), axes=([1, 2, 3, 4], [0, 1, 2, 3])
        ).sum(axis=1)
        self.F_2d = np.tensordot(
            self.T_2d, self.X_2d, axes=(2, 0)
        )
        self.Y_2d, self.Z_2d = self.fit_co_alive(self.F_2d)
        
    def fit_co_alive(self, F_2d):
        Y_co = np.zeros((F_2d.shape[0]))
        self.alive[:, 0] = self.vilsize - self.n_wolves - 1
        self.alive[:, 1] = self.n_wolves
        self.alive[:, 2] = 1
        self.alive -= (F_2d[:, :, 0] + F_2d[:, :, 1])
        Z_alive = np.ones((F_2d.shape[0]))
        for i in range(F_2d.shape[0]):
            # village Seer would comingout
            if F_2d[i, 2, 2] + F_2d[i, 1, 2] > 0 and F_2d[i, 0, 2] == 0:
                Y_co[i] += 2.0
            # village Medium would comingout
            if F_2d[i, 2, 3] + F_2d[i, 1, 3] > 0 and F_2d[i, 0, 3] == 0:
                Y_co[i] += 2.0
            # village Bodyguard would comingout
            if F_2d[i, 2, 4] + F_2d[i, 1, 4] > 0 and F_2d[i, 0, 4] == 0:
                Y_co[i] += 2.0
            # possessed would comingout
            if F_2d[i, 2, 2] + F_2d[i, 2, 3] + F_2d[i, 2, 4] == 0:
                Y_co[i] += 1.0
            # werewolves might not comingout
            Y_co[i] += (F_2d[i, 2, 2] + F_2d[i, 2, 3] + F_2d[i, 2, 4]) * 0
            # villagers must not comingout
            if F_2d[i, 0, 2] > 1:
                Y_co[i] += 3.0
            if F_2d[i, 0, 3] > 1:
                Y_co[i] += 3.0
            if F_2d[i, 0, 4] > 1:
                Y_co[i] += 3.0
            # werewolves are never to be attacked
            if F_2d[i, 1, 1] > 0:
                Z_alive[i] *= 0
            # game is going on, there's at least one werewolf
            if self.alive[i, 1] <= 0:
                Z_alive[i] *= 0
            # game is going on, humans > wolves
            if self.alive[i, 1] >= self.alive[i, 0] + self.alive[i, 2]:
                Z_alive[i] *= 0
        return -np.log(10)*Y_co, Z_alive
    
    # to return 1
    def prob_now(self):
        """return [P(agent_idx = VILLAGER), P(agent_idx = POSSESSED), P(agent_idx = WEREWOLF)]
        """
        return np.tensordot(self.T_2d, 
                            self.Z_2d*np.exp(self.Y_3d + self.Y_2d) / np.sum(self.Z_2d*np.exp(self.Y_3d + self.Y_2d)), 
                            axes=[0, 0]).transpose((1,0))
    
    
    ####################
    # methods for X_3d #
    ####################
    def get_votelist(self, votelist):
        """get votelist as list of list [[x, y],...]
        x:agent, y:target
        """
        # [i, j, 0] : agent i voted agent j (not in talk, action)
        for vote in votelist:
            self.X_3d[vote[0], vote[1], 0] += 1
            
    def get_divine(self, divine):
        """get a new divine result in form [x, y, race]
        x:agent, y:target, race:0:"HUMAN", 1:"WEREWOLF"
        """
        # [i, j, 1] : agent i divined agent j HUMAN
        # [i, j, 2] : agent i divined agent j WEREWOLF
        self.X_3d[divine[0], divine[1], divine[2] + 1] += 1
    
    def get_inquest(self, inquest):
        """get a new inquest result in form [x, y, race]
        x:agent, y:target, race:0:"HUMAN", 1:"WEREWOLF"
        """
        # [i, j, 3] : agent i inquested agent j HUMAN
        # [i, j, 4] : agent i inquested agent j WEREWOLF
        self.X_3d[inquest[0], inquest[1], inquest[2] + 3] += 1
    
    def get_guard(self, guard):
        """get a new guard result in form [x, y]
        x:agent, y:target
        """
        # [i, j, 5] : agent i guarded agent j
        self.X_3d[guard[0], guard[1], 5] += 1
    
    ####################
    # methods for X_2d #
    ####################
    def get_executed(self, x):
        """get agentid newly executed
        x:agent
        """
        # [i, 0] : agent i is executed
        self.X_2d[x, 0] = 1
            
    def get_attacked(self, x):
        """get agentid newly attacked by werewolves
        x:agent
        """
        # [i, 1] : agent i is attacked
        self.X_2d[x, 1] = 1
            
    def get_co(self, co):
        """get a new co in form [x, role]
        x:agent, role:0:"SEER", 1:"MEDIUM", 2:"BODYGUARD", 3:"VILLAGER", 4:"POSSESSED", 5:"WEREWOLF"
        """
        # [i, 2] : agent i comingout himself/herself SEER
        # [i, 3] : agent i comingout himself/herself MEDIUM
        # [i, 4] : agent i comingout himself/herself BODYGUARD
        # [i, 5] : agent i comingout himself/herself VILLAGER
        # [i, 6] : agent i comingout himself/herself POSSESSED
        # [i, 7] : agent i comingout himself/herself WEREWOLF
        self.X_2d[co[0], co[1] + 2] += 1
    
        



In [4]:
agent_names = ['0:fukuda', '1:arai', '2:takenaka', '3:matsushima', '4:kim', '5:mizuguchi', 
               '6:mizuki', '7:tsunakawa', '8:fuyutsumi', '9:ohama', '10:saito', '11:BOSS']

In [5]:
threearrow_1_1 = EnemyCaseTensor(core_cube_base=core_cube_gat2016, vilsize=12, n_wolves=2)
threearrow_1_1.initialize_T_2d()
threearrow_1_1.initialize_T_4d()
threearrow_1_1.initialize_X()

In [6]:
np.set_printoptions(suppress=True)
def print_matrix(A, agent_names):
    return pd.DataFrame(A, index=agent_names, columns=["村", "人狼", "狂人"])

In [7]:
threearrow_1_1.update_Y()
print_matrix(threearrow_1_1.prob_now(), agent_names)

,村,人狼,狂人
0:fukuda,0.75,0.166667,0.083333
1:arai,0.75,0.166667,0.083333
2:takenaka,0.75,0.166667,0.083333
3:matsushima,0.75,0.166667,0.083333
4:kim,0.75,0.166667,0.083333
5:mizuguchi,0.75,0.166667,0.083333
6:mizuki,0.75,0.166667,0.083333
7:tsunakawa,0.75,0.166667,0.083333
8:fuyutsumi,0.75,0.166667,0.083333
9:ohama,0.75,0.166667,0.083333


In [8]:
threearrow_1_1.get_co([7, 0])
threearrow_1_1.get_divine([7, 1, 0])
threearrow_1_1.get_co([9, 0])
threearrow_1_1.get_divine([9, 7, 0])
threearrow_1_1.get_votelist([[11, 7], [10, 2], [9, 2], [8, 0], [7, 2], [6, 2], 
                            [5, 0], [4, 6], [3, 7], [2, 7], [1, 7], [0, 7]])
threearrow_1_1.get_executed(7)

In [9]:
threearrow_1_1.update_Y()
print_matrix(threearrow_1_1.prob_now(), agent_names)

,村,人狼,狂人
0:fukuda,0.830575,0.159109,0.010316
1:arai,0.926542,0.059782,0.013676
2:takenaka,0.860090,0.133987,0.005923
3:matsushima,0.783428,0.204562,0.012009
4:kim,0.764481,0.223682,0.011837
5:mizuguchi,0.756964,0.231198,0.011838
6:mizuki,0.785228,0.203631,0.011141
7:tsunakawa,0.628970,0.001746,0.369284
8:fuyutsumi,0.756964,0.231198,0.011838
9:ohama,0.369192,0.112667,0.518141


In [10]:
threearrow_1_1.get_attacked(5)
threearrow_1_1.get_divine([9, 2, 0])
threearrow_1_1.get_votelist([[11, 10], [10, 6], [9, 6], [8, 6], [6, 11], 
                            [4, 1], [3, 6], [2, 6], [1, 8], [0, 10]])

threearrow_1_1.get_executed(6)

In [11]:
threearrow_1_1.update_Y()
print_matrix(threearrow_1_1.prob_now(), agent_names)

,村,人狼,狂人
0:fukuda,0.753200,0.232571,0.014228
1:arai,0.923793,0.058938,0.017268
2:takenaka,0.847400,0.144021,0.008579
3:matsushima,0.705920,0.278026,0.016054
4:kim,0.665755,0.318209,0.016036
5:mizuguchi,0.981131,0.000000,0.018869
6:mizuki,0.906904,0.086959,0.006137
7:tsunakawa,0.700583,0.002040,0.297377
8:fuyutsumi,0.707703,0.276579,0.015718
9:ohama,0.296820,0.142010,0.561170


In [12]:
threearrow_1_1.get_attacked(11)
threearrow_1_1.get_divine([9, 10, 0])
threearrow_1_1.get_co([10, 0])
threearrow_1_1.get_inquest([10, 7, 0])
threearrow_1_1.get_inquest([10, 6, 0])
threearrow_1_1.get_votelist([[10, 1], [9, 1], [8, 1], [4, 3], [3, 1], [2, 1], [1, 8], [0, 1]])
threearrow_1_1.get_executed(1)

In [13]:
threearrow_1_1.update_Y()
print_matrix(threearrow_1_1.prob_now(), agent_names)

,村,人狼,狂人
0:fukuda,0.889256,0.102322,0.008422
1:arai,0.992798,0.004835,0.002366
2:takenaka,0.976504,0.021111,0.002385
3:matsushima,0.758696,0.231960,0.009344
4:kim,0.710363,0.280272,0.009365
5:mizuguchi,0.990606,0.000000,0.009394
6:mizuki,0.980760,0.016860,0.002380
7:tsunakawa,0.704216,0.006507,0.289277
8:fuyutsumi,0.710329,0.280307,0.009365
9:ohama,0.005267,0.688358,0.306375


In [14]:
threearrow_1_1.get_divine([9, 3, 0])
threearrow_1_1.get_inquest([10, 1, 0])
threearrow_1_1.get_votelist([[10, 8], [9, 8], [8, 0], [4, 8], [3, 8], [2, 8], [0, 8]])
threearrow_1_1.get_executed(8)

In [15]:
threearrow_1_1.update_Y()
print_matrix(threearrow_1_1.prob_now(), agent_names)

,村,人狼,狂人
0:fukuda,0.849234,0.139302,0.011463
1:arai,0.996432,0.000353,0.003216
2:takenaka,0.968116,0.028677,0.003207
3:matsushima,0.672324,0.314954,0.012723
4:kim,0.605557,0.381709,0.012733
5:mizuguchi,0.987241,0.000000,0.012759
6:mizuki,0.973806,0.022966,0.003228
7:tsunakawa,0.720832,0.006169,0.272999
8:fuyutsumi,0.961362,0.035418,0.003220
9:ohama,0.003258,0.695075,0.301667


In [16]:
threearrow_1_1.get_attacked(2)
threearrow_1_1.get_divine([9, 4, 1])
threearrow_1_1.get_inquest([10, 8, 0])
threearrow_1_1.get_co([4, 2])

In [17]:
threearrow_1_1.update_Y()
print_matrix(threearrow_1_1.prob_now(), agent_names)

,村,人狼,狂人
0:fukuda,0.734778,0.244999,0.020222
1:arai,0.993821,0.000512,0.005667
2:takenaka,0.994372,0.000000,0.005628
3:matsushima,0.421455,0.556101,0.022445
4:kim,0.996361,0.002515,0.001124
5:mizuguchi,0.977513,0.000000,0.022487
6:mizuki,0.954088,0.040228,0.005684
7:tsunakawa,0.768608,0.006138,0.225254
8:fuyutsumi,0.954098,0.040225,0.005677
9:ohama,0.000057,0.710383,0.289560
